In [ ]:
from lib.solver_scheduler import solver_registry, scheduler_registry
from lib.cacher_quantizer import cacher_quantizer_registry
import torch
from diffusers import AutoencoderKL


def load_pipe(solver, scheduler, cacher_quantizer):
  PipeClass      = cacher_quantizer_registry[cacher_quantizer]
  SolverClass    = solver_registry[solver]
  SchedulerClassMixin = scheduler_registry[scheduler]

  pipe = PipeClass.from_pretrained()
  class SolverSchedulerConstructor(SolverClass, SchedulerClassMixin):
    def set_timesteps(self, *args, **kwargs):
      SchedulerClassMixin.set_timesteps(self, *args, **kwargs)
  pipe.scheduler = SolverSchedulerConstructor.from_config()

  pipe = pipe.to('cuda')

  pipe.vae = AutoencoderKL.from_pretrained(
    'madebyollin/sdxl-vae-fp16-fix',
    use_safetensors=True,
    torch_dtype=torch.float16,
  ).to('cuda')
  return pipe

In [ ]:
pipe = load_pipe(
  solver           = 'DEIS', 
  scheduler        = 'AYS',
  cacher_quantizer = 'DEEPCACHE'
  )


pipe(
  'A frog in a suit giving a motivational speech at a pond, to an audience of ducks', 
  guidance_scale=5,
  num_inference_steps = 80,
  generator = torch.Generator(device='cuda').manual_seed(42),
  return_dict=False,
  )[0][0]